# Breast Cancer Classification

In [ ]:
# make connection to google drive to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import relevant libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

%matplotlib inline

## Accessing Data Files in Google Drive

In [ ]:
# # set up directory paths to image files
# train_path = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/training_data/fold_0"
# test_path = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data"
# val_path = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/validation_data/val_data"

In [ ]:
combined_path_train = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT/combined_images_train"

In [ ]:
normal_imgs = [] # store normal cell image names
cancer_imgs = [] # store cancer cell image names

for file in os.listdir(combined_path_train+"/normal"): # normal cells
  if 'mask' not in file:
    normal_imgs.append(file)
for file in os.listdir(combined_path_train+"/malignant"): # cancer cells
  if 'mask' not in file:
    cancer_imgs.append(file)

In [ ]:
len(normal_imgs), len(cancer_imgs)

In [ ]:
image_path = os.path.join(combined_path_train+"/normal",normal_imgs[2]) # create the full path to the image
img = mpimg.imread(image_path) # read the image data
print(img.shape) # print the shape of the image data
plt.imshow(img) # display the image data
plt.title("Normal Image", fontsize=14, fontweight='bold')
plt.show()

In [ ]:
image_path = os.path.join(combined_path_train+"/malignant", cancer_imgs[2]) # create the full path to the image
img = mpimg.imread(image_path) # read the image data
print(img.shape) # print the shape of the image data
plt.imshow(img) # display the image data
plt.title("Cancer Image", fontsize=14, fontweight='bold')
plt.show()

## Image Preprocessing

In [ ]:
dim1 = [] # storing first dimensions of image for analysis
dim2 = [] # storing second dimensions of image for analysis

for image_filename in os.listdir(combined_path_train+'/normal'): # parse through all non-cancer images
    if 'mask' not in image_filename:
      img = mpimg.imread(combined_path_train+'/normal'+'/'+image_filename) # convert image to array
      d1,d2,colors = img.shape # get dimensions of image
      dim1.append(d1) # append to dimension 1 list
      dim2.append(d2) # append to dimension 2 list

for image_filename in os.listdir(combined_path_train+'/malignant'): # parse through all cancer images
    if 'mask' not in image_filename:
      img = mpimg.imread(combined_path_train+'/malignant'+'/'+image_filename) # convert image to array
      d1,d2,colors = img.shape # get dimensions of image
      dim1.append(d1) # append to dimension 1 list
      dim2.append(d2) # append to dimension 2 list

In [ ]:
# check for variations in dimensions
np.mean(dim1),np.mean(dim2)

In [ ]:
# standard shape of image
image_shape = (226,226,1)

In [ ]:
# import library for image manipulation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def preprocess_image(x):
    if 'mask' not in x:
      return x
    else:
      return None

In [ ]:
# set up generator for image manipulation to add diversity in dataset
image_gen = ImageDataGenerator(rotation_range=20, # rotate 20 degrees
                               width_shift_range=0.10, # shift width by max of 5%
                               height_shift_range=0.10, # shift height by max of 5%
                               rescale=1/255, # rescale image by normalzing
                               shear_range=0.1, # cutting part of image (max 10%)
                               zoom_range=0.1, # zoom by 10% max
                               horizontal_flip=True, # horizontal flip
                               fill_mode='nearest', # fill missing pixels with nearest filled value
                               preprocessing_function=preprocess_image # preprocess image
                              )

In [ ]:
# rerun this cell as an example of how image is transformed
plt.imshow(image_gen.random_transform(img))

## Convolutional Neural Network Model for Image Classification

In [ ]:
# import relevant libraries from tensor flow for model construction
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()  # make sequential model

# add convolutional layers with pooling layers (max pooling)
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten output
model.add(Flatten())

# add batch normalization
model.add(BatchNormalization())

# add layer of neurons with 'relu' activation
model.add(Dense(128))
model.add(Activation('relu'))

# add dropout layer, rate=0.5
model.add(Dropout(0.6))

# final output neuron for prediction (cancer or not cancer) with 'sigmoid' activation
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compile model with 'ADAM' optimizer
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# prints out model summary
model.summary()

In [ ]:
# early stopping to monitor validation loss for no improvement
early_stop = EarlyStopping(monitor='loss',patience=5)

In [ ]:
batch_size = 32 # to train in batches

In [ ]:
# generator to get training images from combined path
train_image_gen = image_gen.flow_from_directory(combined_path_train,
                                                target_size=(224, 224),
                                                color_mode='grayscale',
                                                batch_size=batch_size,
                                                class_mode='binary')

In [ ]:
train_image_gen.class_indices # check labels of data

In [ ]:
# suppress any warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# fit the model on dataset
results = model.fit(train_image_gen,epochs=20,callbacks=[early_stop])

In [ ]:
# # save entire CNN model
# model.save('cnn_balanced_model.keras')

In [ ]:
# # save weights separately
# model.save_weights('cnn_balanced_weights.weights.h5')

In [ ]:
# # import relevant library to store model history as json file
# import json
# with open('cnn_balanced_history.json', 'w') as f:
#     json.dump(results.history, f)

In [ ]:
# # save model architecture to JSON
# with open("cnn_balanced_architecture.json", "w") as json_file:
#     json_file.write(model.to_json())

## Model Evaluation

In [ ]:
# store results as dataframe
losses = pd.DataFrame(model.history.history)

In [ ]:
# plot training and validation losses
ax = losses[['loss']].plot()

# set title and labels
ax.set_title('Training Losses', fontsize=16, fontweight='bold')
ax.set_xlabel('Epochs', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)

plt.show()

In [ ]:
# plot training and validation accuracies
ax = losses[['accuracy']].plot()

# set title and labels
ax.set_title('Training Accuracies', fontsize=16, fontweight='bold')
ax.set_xlabel('Epochs', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)

plt.show()

In [ ]:
# metrics that were analyzed during fitting
model.metrics_names

In [ ]:
# example evaluation on validation dataset to get those metrics
model.evaluate(train_image_gen)

In [ ]:
# predict on validation dataset
pred_probabilities = model.predict(train_image_gen)

In [ ]:
# classify predictions in binary method
predictions = pred_probabilities > 0.5
# predictions = (pred > 0.5).astype('int') # another way to do it

In [ ]:
train_image_gen.classes # check labels of data

In [ ]:
# import relevant libraries for classification metrics and quantitative analysis
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# print classification report to understand key metrics
print(classification_report(train_image_gen.classes,predictions))

In [ ]:
# print confusion matrix to investigate accurate predictions
print(confusion_matrix(train_image_gen.classes,predictions))

In [ ]:
# get confusion matrix
cm = confusion_matrix(train_image_gen.classes,predictions)

# extract True Positives, False Negatives, False Positives, True Negatives
TN, FP, FN, TP = cm.ravel()

# calculate sensitivity and specificity
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# calculate AUC-ROC score
auc_roc = roc_auc_score(train_image_gen.classes,predictions)
print(f"AUC-ROC Score: {auc_roc:.8f}")

# compute ROC curve
fpr, tpr, thresholds = roc_curve(train_image_gen.classes,predictions)

# plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='green', lw=2, label=f'ROC curve (AUC = {auc_roc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate',fontsize=12)
plt.ylabel('True Positive Rate',fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve',fontsize=14,fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Prediction on Testing Data

In [ ]:
# import relevant library for image analysis
from tensorflow.keras.preprocessing import image

In [ ]:
train_imgs = [] # list to store all training image names

for file in os.listdir(combined_path_train+'/normal'): # parse through training directory and get all normal cell images
    train_imgs.append(file)

In [ ]:
# load an example image - can change number
my_image = image.load_img(os.path.join(combined_path_train+'/normal',train_imgs[3]),target_size=image_shape,color_mode='grayscale')

In [ ]:
# plot to see the image of cell
plt.imshow(my_image)

In [ ]:
my_image = image.img_to_array(my_image) # convert from image file to array for prediction

In [ ]:
my_image_array = np.expand_dims(my_image, axis=0)  # Shape becomes (1, 224, 224, 1)

In [ ]:
my_image_array.shape # check shape of image array

In [ ]:
model.predict(my_image_array) # make prediction on current selected cell - should be normal

In [ ]:
train_image_gen.class_indices # definition of indices